<a href="https://colab.research.google.com/github/Tara2712/Informatika/blob/main/Procesiranje_podatkov/dokon%C4%8Dna_predpriprava_podatkov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [194]:
import pandas as pd

In [195]:
df = pd.read_csv('opis_cleaned.csv', index_col='SR')

In [196]:
df.shape

(28353, 6)

In [197]:
# df.head()

In [198]:
df.isna().sum()

,0
NAZIV_SR,0
DATUM_NASTANKA_SR,0
STATUS,0
WORKLOGID,900
OPIS,2499
DOLGI_OPIS_X,10210


In [199]:
def clean_and_null(text):
    if not isinstance(text, str):
        return np.nan
    cleaned = re.sub(r'img\s+[^>]*?src="data:image[^"]+"', '', text, flags=re.IGNORECASE).strip()
    return cleaned if cleaned else np.nan

df["DOLGI_OPIS_X"] = df["DOLGI_OPIS_X"].apply(clean_and_null)
# df.iloc[87:91]

In [200]:
df.isna().sum()

,0
NAZIV_SR,0
DATUM_NASTANKA_SR,0
STATUS,0
WORKLOGID,900
OPIS,2499
DOLGI_OPIS_X,10281


In [201]:
stolpci_za_preverit = ["OPIS", "DOLGI_OPIS_X"]

num_both_null = df[(df['OPIS'].isna()) & (df['DOLGI_OPIS_X'].isna())].shape[0] #& (df['WORKLOGID'].isna())
print(f"Number of rows where both 'OPIS' and 'DOLGI_OPIS_X' are null: {num_both_null}")

Number of rows where both 'OPIS' and 'DOLGI_OPIS_X' are null: 1297


In [202]:
df_pociscen = df.dropna(subset=stolpci_za_preverit, how="all")

In [203]:
df_pociscen.isna().sum()

,0
NAZIV_SR,0
DATUM_NASTANKA_SR,0
STATUS,0
WORKLOGID,0
OPIS,1202
DOLGI_OPIS_X,8984


In [204]:
def drop_empty_resitev(df):
    pogoj = ((df["OPIS"] == "Rešitev") | (df["OPIS"] == "Potrjujem rešitev") | (df["OPIS"] == "Zavračam rešitev")) & (df["DOLGI_OPIS_X"].isnull() | (df["DOLGI_OPIS_X"].str.strip() == ""))
    df_cleaned = df[~pogoj].copy()  # ~ znegira pogoj
    return df_cleaned

In [205]:
df_pociscen = drop_empty_resitev(df_pociscen)

In [206]:
df_pociscen.isna().sum()

,0
NAZIV_SR,0
DATUM_NASTANKA_SR,0
STATUS,0
WORKLOGID,0
OPIS,1202
DOLGI_OPIS_X,518


In [207]:
df_pociscen['OPIS'].value_counts()

,count
OPIS,
Rešitev,10907
Komentar IN,279
Komentar INF,252
Odgovor,217
Zahtevek sprejet,172
...,...
Izpisek s tem datumom že obstaja,1
Zapis pri fakturi v delovnem toku,1
Odgovor informatika,1


In [208]:
# df_pociscen.head()

In [209]:
def drop_empty_zavracam_resitev(df):
    pogoj = ((df["OPIS"] == "Zavračam rešitev") | (df["OPIS"] == "Zapiramo zahtevek")| (df["OPIS"] == "Zahtevek zapiramo") | (df["OPIS"] == "Urejeno")) & (df["DOLGI_OPIS_X"].isnull() | (df["DOLGI_OPIS_X"].str.strip() == "") )
    df_cleaned = df[~pogoj].copy()
    return df_cleaned

In [210]:
df_pociscen = drop_empty_zavracam_resitev(df_pociscen)

In [211]:
df_pociscen.isna().sum()

,0
NAZIV_SR,0
DATUM_NASTANKA_SR,0
STATUS,0
WORKLOGID,0
OPIS,1202
DOLGI_OPIS_X,509


In [212]:
df_pociscen.shape

(18581, 6)

In [213]:
df_pociscen["OPIS"].value_counts()

,count
OPIS,
Rešitev,10907
Komentar IN,279
Komentar INF,252
Odgovor,217
Zahtevek sprejet,172
...,...
Zapis pri fakturi v delovnem toku,1
Odgovor informatika,1
sprejeto v reševanje,1


In [214]:
# df_pociscen.to_csv('2proba.csv', index=True)

In [215]:
# df_pociscen = df_pociscen['DOLGI_OPIS_X'].dropna()

In [216]:
df_pociscen.isna().sum()

,0
NAZIV_SR,0
DATUM_NASTANKA_SR,0
STATUS,0
WORKLOGID,0
OPIS,1202
DOLGI_OPIS_X,509


In [217]:
df_pociscen = df_pociscen.dropna(subset=['DOLGI_OPIS_X'])
df_pociscen.isna().sum()

,0
NAZIV_SR,0
DATUM_NASTANKA_SR,0
STATUS,0
WORKLOGID,0
OPIS,1202
DOLGI_OPIS_X,0


In [218]:
df_pociscen.shape

(18072, 6)

In [219]:
from sklearn.impute import SimpleImputer
import numpy as np

In [220]:
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

df_pociscen["OPIS"] = imputer.fit_transform(df_pociscen[["OPIS"]]).ravel()

<ipython-input-220-1c1063f251bb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pociscen["OPIS"] = imputer.fit_transform(df_pociscen[["OPIS"]]).ravel()


In [221]:
df_pociscen.isna().sum()

,0
NAZIV_SR,0
DATUM_NASTANKA_SR,0
STATUS,0
WORKLOGID,0
OPIS,0
DOLGI_OPIS_X,0


In [222]:
df_pociscen.shape

(18072, 6)

In [223]:
df_pociscen.to_csv("df_nan.csv", index=True)

In [224]:
import re

In [225]:
# def remove_base64_images(text):
#     # Remove 'img ... base64,...' style embedded images
#     return re.sub(r'img[^"]*src="data:image[^"]+"', '', text)

# # Apply to the column
# df_pociscen["DOLGI_OPIS_X"] = df_pociscen["DOLGI_OPIS_X"].fillna("").apply(remove_base64_images)

In [226]:
df_pociscen.isna().sum()

,0
NAZIV_SR,0
DATUM_NASTANKA_SR,0
STATUS,0
WORKLOGID,0
OPIS,0
DOLGI_OPIS_X,0


In [227]:
# import re
# import pandas as pd

# def remove_base64_images(text):
#     if not isinstance(text, str):
#         return text

#     return re.sub(r'img\s+[^>]*?src="data:image[^"]+"', '', text, flags=re.IGNORECASE)

# df_pociscen["DOLGI_OPIS_X"] = df_pociscen["DOLGI_OPIS_X"].fillna("").apply(remove_base64_images)


In [228]:
# df_pociscen.head(89)

In [229]:
# df_pociscen.isna().sum()

In [230]:
# def clean_and_nullify(text):
#     if not isinstance(text, str):
#         return np.nan

#     cleaned = re.sub(r'img\s+[^>]*?src="data:image[^"]+"', '', text, flags=re.IGNORECASE)

#     cleaned = cleaned.strip()

#     return cleaned if cleaned else np.nan


# df_pociscen["DOLGI_OPIS_X"] = df_pociscen["DOLGI_OPIS_X"].apply(clean_and_nullify)

In [231]:
# df_pociscen.iloc[87:91]

In [232]:
# import re
# import numpy as np

# def clean_and_nullify(text):
#     if not isinstance(text, str):
#         return np.nan

#     cleaned = re.sub(
#         r'img\s+[^>]*?src=["\']data:image[^"\']+["\']',
#         '',
#         text,
#         flags=re.IGNORECASE
#     )


#     cleaned = cleaned.strip()


#     return cleaned if cleaned else np.nan


# df_pociscen["DOLGI_OPIS_X"] = df_pociscen["DOLGI_OPIS_X"].apply(clean_and_nullify)


In [233]:
df_pociscen.isna().sum()

,0
NAZIV_SR,0
DATUM_NASTANKA_SR,0
STATUS,0
WORKLOGID,0
OPIS,0
DOLGI_OPIS_X,0


In [234]:
# import pandas as pd
# import re

# def clean_cell(text):
#     if not isinstance(text, str):
#         return text

#     text = re.sub(r'<img[^>]+src="data:image[^"]+"[^>]*>', '', text)

#     text = re.sub(r'<[^>]+>', '', text)

#     text = re.sub(r'\s+', ' ', text).strip()
#     return text

# df_pociscen = df_pociscen.applymap(clean_cell)


In [235]:
df_pociscen.isna().sum()

,0
NAZIV_SR,0
DATUM_NASTANKA_SR,0
STATUS,0
WORKLOGID,0
OPIS,0
DOLGI_OPIS_X,0


In [236]:
# import re

# def remove_img_src(content):
#     return re.sub(r'\s*src="[^"]*"', '', content, flags=re.IGNORECASE)

# # Example usage
# html = 'Uspešno testirano, zato prosim za prenos na produkcijo.Hvala. <img alt="" src="data:image/png;base64,iVBOR..." />'
# cleaned_html = remove_img_src(html)

# print(cleaned_html)


In [237]:

# def remove_img_src(content):
#     return re.sub(r'\s*src="[^"]*"', '', content, flags=re.IGNORECASE)


# df_pociscen['DOLGI_OPIS_X'] = df_pociscen['DOLGI_OPIS_X'].astype(str).apply(remove_img_src)

In [238]:
# df_pociscen.to_csv("df_pociscen.csv", index=True)